# Hyperparameters Tuning

Parameters are the components of the model that are learned during the training process and we can never set them manually. A model starts the training process with random parameter values and adjusts them throughout. Whereas, hyperparameters are the components set by you before the training of the model. The values of hyperparameters might improve or worsen your model’s accuracy.

Machine learning models are not intelligent enough to know what hyperparameters would lead to the highest possible accuracy on the given dataset. However, hyperparameter values when set right can build highly accurate models, and thus we allow our models to try different combinations of hyperparameters during the training process and make predictions with the best combination of hyperparameter values.

## Grid Search


In grid search, each square in a grid has a combination of hyperparameters and the model has to train itself on each combination. For a clearer understanding, suppose that we want to train a Random Forest Classifier with the following set of hyperparameters.
* n_estimators: [100, 150, 200]
* max_depth: [20, 30, 40]

Have a look at the grid made from these hyperparameter values. Our model runs the training process on each combination of n_estimators and max_depth
<p align="center">
<img src="GS.png" width="800" height="600" alt="something is wrong">
</p>

## Import Dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
import optuna
import plotly

# Import Dataset

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


## Preprocessing

In [4]:
y = df.pop('price_range')
x_train, x_test, y_train, y_test = train_test_split(df,y, test_size = 0.2)

## Normalization

In [5]:
train_scaler = StandardScaler().fit(x_train)
test_scaler = StandardScaler().fit(x_test)

In [6]:
x_train_scaled = train_scaler.transform(x_train)
x_test_scaled = test_scaler.transform(x_test)

## Random Forest Classification with Default Hyperparameter Values

In [7]:
classifier1 = RandomForestClassifier().fit(x_train_scaled, y_train)
print("Training score is:" + str(classifier1.score(x_train_scaled, y_train)))
print("Test score is:" + str(classifier1.score(x_test_scaled, y_test)))

Training score is:1.0
Test score is:0.87


## Random Forest Classification with Grid Search CV

In [8]:
hyperparameters = {'n_estimators' : (30, 40, 50, 100, 200), 'min_samples_leaf' : (2, 3, 5, 10), 'max_features' : ('auto', 'sqrt', 'log2', None), 'max_depth' : (10, 20, 30, 50, 100)}
rfc = RandomForestClassifier()
classifier2 = GridSearchCV(rfc, hyperparameters, cv=8, scoring='accuracy')
classifier2.fit(x_train_scaled, y_train)

GridSearchCV(cv=8, estimator=RandomForestClassifier(),
             param_grid={'max_depth': (10, 20, 30, 50, 100),
                         'max_features': ('auto', 'sqrt', 'log2', None),
                         'min_samples_leaf': (2, 3, 5, 10),
                         'n_estimators': (30, 40, 50, 100, 200)},
             scoring='accuracy')

In [9]:
print("The best score is : " + str(classifier2.best_score_))
best_rfc = classifier2.best_estimator_
print(best_rfc)
print("Test score is:" + str(best_rfc.score(x_test_scaled, y_test)))

The best score is : 0.90375
RandomForestClassifier(max_depth=30, max_features=None, min_samples_leaf=3)
Test score is:0.8625


## Random Search

Like grid search, we still set the hyperparameter values we want to tune in Random Search. However, the model does not train each combination of hyperparameters, it instead selects them randomly. We have to define the number of samples we want to choose from our grid, So it is computationally more efficient to find best combination of hyperparameters.

## Random Forest Classification with Random Search

In [10]:
hyperparameters2 = {'n_estimators' : (30, 40, 50, 100, 200, 300), 'min_samples_leaf' : (2, 3, 5, 10, 20), 'max_features' : ('auto', 'sqrt', 'log2', None), 'max_depth' : (10, 20, 30, 50, 100, 200)}
rfc2 = RandomForestClassifier()
classifier3 = RandomizedSearchCV(rfc2, hyperparameters2, n_iter=100, cv=8, scoring='accuracy')
classifier3.fit(x_train_scaled, y_train)

RandomizedSearchCV(cv=8, estimator=RandomForestClassifier(), n_iter=100,
                   param_distributions={'max_depth': (10, 20, 30, 50, 100, 200),
                                        'max_features': ('auto', 'sqrt', 'log2',
                                                         None),
                                        'min_samples_leaf': (2, 3, 5, 10, 20),
                                        'n_estimators': (30, 40, 50, 100, 200,
                                                         300)},
                   scoring='accuracy')

In [11]:
print("The best score is : " + str(classifier3.best_score_))
best_rfc2 = classifier3.best_estimator_
print(best_rfc2)
print("Test score is:" + str(best_rfc2.score(x_test_scaled, y_test)))

The best score is : 0.899375
RandomForestClassifier(max_depth=20, max_features=None, min_samples_leaf=3,
                       n_estimators=300)
Test score is:0.86


## Optuna

Optuna is an open-source hyperparameter optimization framework to automate hyperparameter search. The key features of Optuna include automated search for optimal hyperparameters, efficiently search large spaces and prune unpromising trials for faster results, and parallelize hyperparameter searches over multiple threads or processes.

In [12]:
def objective(trial):
    n_estimators = int(trial.suggest_loguniform('n_estimators', 5, 300))
    max_depth = int(trial.suggest_loguniform('max_depth', 10, 500))
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20)
    classifier4 = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    scores = cross_val_score(classifier4, x_train_scaled, y_train, cv=8)
    score = scores.mean()
    return score

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
best_trial = study.best_trial
print('Accuracy: {}'.format(best_trial.value))
print("Best hyperparameters: {}".format(best_trial.params))

[I 2022-05-06 16:54:14,649] A new study created in memory with name: no-name-4a2d7bdc-76f6-4e72-a010-cfbbf2f9f9b3
[I 2022-05-06 16:54:14,799] Trial 0 finished with value: 0.76875 and parameters: {'n_estimators': 6.620750116799072, 'max_depth': 84.45707822726965, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.76875.
[I 2022-05-06 16:54:15,770] Trial 1 finished with value: 0.865 and parameters: {'n_estimators': 42.08678291709898, 'max_depth': 298.02920238111227, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.865.
[I 2022-05-06 16:54:16,023] Trial 2 finished with value: 0.79875 and parameters: {'n_estimators': 13.56205001442954, 'max_depth': 15.428539617780112, 'min_samples_leaf': 14}. Best is trial 1 with value: 0.865.
[I 2022-05-06 16:54:16,506] Trial 3 finished with value: 0.846875 and parameters: {'n_estimators': 23.80451941571961, 'max_depth': 213.61951113344585, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.865.
[I 2022-05-06 16:54:16,639] Trial 4 finished with

Accuracy: 0.88625
Best hyperparameters: {'n_estimators': 106.98335337738521, 'max_depth': 39.64852544538011, 'min_samples_leaf': 2}


In [14]:
optuna.visualization.plot_optimization_history(study)

In [15]:
optuna.visualization.plot_slice(study)